In [1]:
#import dependancies 
import evalml
from evalml import AutoMLSearch
from evalml.objectives import FraudCost

c:\Users\zirot\anaconda3\envs\evalml\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
#set our own assumptions for the cost of fraud.
#EvalML determines attempt to build a pipeline
#that will minimize the financial loss due to fraud.
fraud_objective = FraudCost(
    retry_percentage=0.5,
    interchange_fee=0.02,
    fraud_payout_percentage=0.75,
    amount_col="amount",
)

In [3]:
X, y = evalml.demos.load_fraud(n_rows=5000)

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 5000
Targets
False    86.20%
True     13.80%
Name: fraud, dtype: object


In [6]:
#Dropping columns and splitting data to also hold data on the side.
cols_to_drop = ["datetime", "expiration_date", "country", "region", "provider"]
for col in cols_to_drop:
    X.ww.pop(col)

X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(
    X, y, problem_type="binary", test_size=0.2, random_seed=0
)

X.ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
card_id,int64,Integer,['numeric']
store_id,int64,Integer,['numeric']
amount,int64,Integer,['numeric']
currency,category,Categorical,['category']
customer_present,bool,Boolean,[]
lat,float64,Double,['numeric']
lng,float64,Double,['numeric']


In [7]:
#Searching for the best pipeline
automl = AutoMLSearch(
    X_train=X_train,
    y_train=y_train,
    problem_type="binary",
    objective=fraud_objective,
    additional_objectives=["auc", "f1", "precision"],
    allowed_model_families=["random_forest", "linear_model"],
    max_batches=1,
    optimize_thresholds=True,
    verbose=True,
)

automl.search(interactive_plot=False)

AutoMLSearch will use mean CV score to rank pipelines.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Cost. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of None pipelines. 
Allowed model families: 

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.790

*****************************
* Evaluating Batch Number 1 *
*****************************

Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model:
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.009

Search finished after 5.12 seconds     
Best pipeline: Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model


{1: {'Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model': 3.1286752223968506,
  'Total time of batch': 3.457847833633423}}

In [8]:
automl.rankings

,id,pipeline_name,search_order,ranking_score,mean_cv_score,standard_deviation_cv_score,percent_better_than_baseline,high_variance_cv,parameters
0,1,Random Forest Classifier w/ Label Encoder + Im...,1,0.008649,0.008649,0.000789,78.099995,False,"{'Label Encoder': {'positive_label': None}, 'I..."
1,0,Mode Baseline Binary Classification Pipeline,0,0.789648,0.789648,0.001136,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [9]:
#select best pipeline
best_pipeline = automl.best_pipeline

In [11]:
# We can get more details about any pipeline created 
# during the search process, including how it performed on 
# other objective functions, by calling the describe_pipeline 
# method and passing the id of the pipeline of interest
automl.describe_pipeline(automl.rankings.iloc[1]["id"])


************************************************
* Mode Baseline Binary Classification Pipeline *
************************************************

Problem Type: binary
Model Family: Baseline

Pipeline Steps
1. Label Encoder
	 * positive_label : None
2. Baseline Classifier
	 * strategy : mode

Training
Training for binary problems.
Objective to optimize binary classification pipeline thresholds for: <evalml.objectives.fraud_cost.FraudCost object at 0x0000025A02ABE550>
Total training time (including CV): 0.7 seconds

Cross Validation
----------------
             Fraud Cost   AUC    F1  Precision # Training # Validation
0                 0.791 0.500 0.000      0.000      2,666        1,334
1                 0.789 0.500 0.000      0.000      2,667        1,333
2                 0.789 0.500 0.000      0.000      2,667        1,333
mean              0.790 0.500 0.000      0.000          -            -
std               0.001 0.000 0.000      0.000          -            -
coef of var      

In [12]:
#best pipeline is already trained, we evaluate it on the holdout data
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", fraud_objective])


OrderedDict([('AUC', 0.8673290964726453),
             ('Fraud Cost', 0.008257252890414273)])

In [13]:
#search for another pipeline using AUC, 
# a common machine learning metric
automl_auc = AutoMLSearch(
    X_train=X_train,
    y_train=y_train,
    problem_type="binary",
    objective="auc",
    additional_objectives=["f1", "precision"],
    max_batches=1,
    allowed_model_families=["random_forest", "linear_model"],
    optimize_thresholds=True,
    verbose=True,
)

automl_auc.search(interactive_plot=False)

AutoMLSearch will use mean CV score to rank pipelines.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of None pipelines. 
Allowed model families: 

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean AUC: 0.500

*****************************
* Evaluating Batch Number 1 *
*****************************

Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model:
	Starting cross validation
	Finished cross validation - mean AUC: 0.852

Search finished after 4.81 seconds     
Best pipeline: Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model
Best pipeline AUC: 

{1: {'Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model': 3.5370755195617676,
  'Total time of batch': 3.880744218826294}}

In [14]:
automl_auc.rankings

,id,pipeline_name,search_order,ranking_score,mean_cv_score,standard_deviation_cv_score,percent_better_than_baseline,high_variance_cv,parameters
0,1,Random Forest Classifier w/ Label Encoder + Im...,1,0.852091,0.852091,0.004235,35.20905,False,"{'Label Encoder': {'positive_label': None}, 'I..."
1,0,Mode Baseline Binary Classification Pipeline,0,0.500000,0.500000,0.000000,0.00000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [15]:
best_pipeline_auc = automl_auc.best_pipeline

In [16]:
# get the fraud score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout, objectives=["auc", fraud_objective])

OrderedDict([('AUC', 0.8673290964726453),
             ('Fraud Cost', 0.025729330840169453)])

In [17]:
# fraud score on fraud optimized again
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", fraud_objective])

OrderedDict([('AUC', 0.8673290964726453),
             ('Fraud Cost', 0.008257252890414273)])